In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [30]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, roc_auc_score

In [2]:
df = pd.read_csv('/home/ubuntu/MIMICmaterialized/oasis.csv')

In [4]:
df.describe(include='all')

,subject_id,hadm_id,icustay_id,icustay_age_group,hospital_expire_flag,icustay_expire_flag,oasis,oasis_prob,age,age_score,...,resprate,resprate_score,temp,temp_score,urineoutput,urineoutput_score,mechvent,mechvent_score,electivesurgery,electivesurgery_score
count,61532.000000,61532.000000,61532.000000,61532,61532.000000,61532.000000,61532.000000,61532.000000,61532.000000,61532.000000,...,52229.000000,52229.000000,51382.000000,51382.000000,53201.000000,53201.000000,61532.000000,61532.000000,61532.000000,61532.000000
unique,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,adult,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,53431,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,33888.605912,149954.470649,249962.710248,NaN,0.107408,0.075099,29.650475,0.130883,64.470552,4.997741,...,24.687757,2.466446,36.463895,2.714161,1820.675878,3.526005,0.400978,3.608805,0.107408,5.355555
std,28127.690913,28898.895904,28890.574867,NaN,0.309633,0.263553,9.811180,0.138882,56.913254,2.820875,...,9.816155,2.661418,1.248250,1.057916,2842.279356,3.663245,0.490101,4.410906,0.309633,1.857799
min,2.000000,100001.000000,200001.000000,NaN,0.000000,0.000000,0.000000,0.002077,0.000000,0.000000,...,0.200000,0.000000,15.000000,0.000000,-2600.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12047.500000,124933.750000,224935.500000,NaN,0.000000,0.000000,23.000000,0.037612,44.000000,3.000000,...,19.000000,1.000000,35.666667,2.000000,876.000000,1.000000,0.000000,0.000000,0.000000,6.000000
50%,24280.500000,149911.500000,249940.000000,NaN,0.000000,0.000000,29.000000,0.077479,62.000000,6.000000,...,26.000000,1.000000,36.111111,2.000000,1580.000000,1.000000,0.000000,0.000000,0.000000,6.000000
75%,54191.500000,174997.250000,274972.500000,NaN,0.000000,0.000000,36.000000,0.170145,76.000000,6.000000,...,30.000000,6.000000,37.388889,4.000000,2470.000000,5.000000,1.000000,9.000000,0.000000,6.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61532 entries, 0 to 61531
Data columns (total 28 columns):
subject_id               61532 non-null int64
hadm_id                  61532 non-null int64
icustay_id               61532 non-null int64
icustay_age_group        61532 non-null object
hospital_expire_flag     61532 non-null int64
icustay_expire_flag      61532 non-null int64
oasis                    61532 non-null int64
oasis_prob               61532 non-null float64
age                      61532 non-null int64
age_score                61532 non-null int64
preiculos                61532 non-null object
preiculos_score          61532 non-null int64
gcs                      52143 non-null float64
gcs_score                52143 non-null float64
heartrate                59048 non-null float64
heartrate_score          59048 non-null float64
meanbp                   52288 non-null float64
meanbp_score             52288 non-null float64
resprate                 52229 non-null float64

In [6]:
df_adult = df.loc[df['icustay_age_group'] == 'adult']

In [7]:
df_adult.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53431 entries, 0 to 61531
Data columns (total 28 columns):
subject_id               53431 non-null int64
hadm_id                  53431 non-null int64
icustay_id               53431 non-null int64
icustay_age_group        53431 non-null object
hospital_expire_flag     53431 non-null int64
icustay_expire_flag      53431 non-null int64
oasis                    53431 non-null int64
oasis_prob               53431 non-null float64
age                      53431 non-null int64
age_score                53431 non-null int64
preiculos                53431 non-null object
preiculos_score          53431 non-null int64
gcs                      52142 non-null float64
gcs_score                52142 non-null float64
heartrate                52293 non-null float64
heartrate_score          52293 non-null float64
meanbp                   52287 non-null float64
meanbp_score             52287 non-null float64
resprate                 52228 non-null float64

In [10]:
df_adult.describe(include='all')

,subject_id,hadm_id,icustay_id,icustay_age_group,hospital_expire_flag,icustay_expire_flag,oasis,oasis_prob,age,age_score,...,resprate,resprate_score,temp,temp_score,urineoutput,urineoutput_score,mechvent,mechvent_score,electivesurgery,electivesurgery_score
count,53431.000000,53431.000000,53431.000000,53431,53431.000000,53431.000000,53431.000000,53431.000000,53431.000000,53431.000000,...,52228.000000,52228.000000,51381.000000,51381.000000,49729.000000,49729.000000,53431.000000,53431.000000,53431.000000,53431.000000
unique,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,adult,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,53431,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,36653.934570,149971.670996,250015.705845,NaN,0.122494,0.085288,31.021916,0.143038,74.245064,5.755479,...,24.687751,2.466474,36.463870,2.714175,1941.832105,3.074182,0.428197,3.853774,0.123692,5.257847
std,28987.770125,28930.407384,28910.150685,NaN,0.327859,0.279312,9.137619,0.143596,54.813507,2.191481,...,9.816248,2.661435,1.248250,1.057921,2901.274107,3.350603,0.494822,4.453399,0.329233,1.975399
min,3.000000,100001.000000,200001.000000,NaN,0.000000,0.000000,0.000000,0.002077,15.000000,0.000000,...,0.200000,0.000000,15.000000,0.000000,-2600.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13215.500000,124880.000000,224951.000000,NaN,0.000000,0.000000,25.000000,0.048012,52.000000,3.000000,...,19.000000,1.000000,35.666667,2.000000,1020.000000,1.000000,0.000000,0.000000,0.000000,6.000000
50%,26579.000000,150011.000000,250016.000000,NaN,0.000000,0.000000,30.000000,0.087098,65.000000,6.000000,...,26.000000,1.000000,36.111111,2.000000,1675.000000,1.000000,0.000000,0.000000,0.000000,6.000000
75%,59889.000000,175054.500000,275072.000000,NaN,0.000000,0.000000,37.000000,0.188911,77.000000,6.000000,...,30.000000,6.000000,37.388889,4.000000,2550.000000,5.000000,1.000000,9.000000,0.000000,6.000000


In [12]:
df_adult_dropped = df_adult.dropna().copy()

In [13]:
df_adult_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48785 entries, 0 to 61531
Data columns (total 28 columns):
subject_id               48785 non-null int64
hadm_id                  48785 non-null int64
icustay_id               48785 non-null int64
icustay_age_group        48785 non-null object
hospital_expire_flag     48785 non-null int64
icustay_expire_flag      48785 non-null int64
oasis                    48785 non-null int64
oasis_prob               48785 non-null float64
age                      48785 non-null int64
age_score                48785 non-null int64
preiculos                48785 non-null object
preiculos_score          48785 non-null int64
gcs                      48785 non-null float64
gcs_score                48785 non-null float64
heartrate                48785 non-null float64
heartrate_score          48785 non-null float64
meanbp                   48785 non-null float64
meanbp_score             48785 non-null float64
resprate                 48785 non-null float64

In [14]:
df_adult_dropped.describe(include='all')

,subject_id,hadm_id,icustay_id,icustay_age_group,hospital_expire_flag,icustay_expire_flag,oasis,oasis_prob,age,age_score,...,resprate,resprate_score,temp,temp_score,urineoutput,urineoutput_score,mechvent,mechvent_score,electivesurgery,electivesurgery_score
count,48785.000000,48785.000000,48785.000000,48785,48785.000000,48785.000000,48785.000000,48785.000000,48785.000000,48785.000000,...,48785.000000,48785.000000,48785.000000,48785.000000,48785.000000,48785.000000,48785.000000,48785.000000,48785.000000,48785.000000
unique,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,adult,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,48785,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,37026.394384,149943.232776,250044.560029,NaN,0.116778,0.080701,31.558184,0.148383,74.264364,5.746910,...,24.678115,2.450548,36.468464,2.713047,1947.167444,3.064159,0.436302,3.926719,0.123214,5.260715
std,29099.517506,28902.844359,28922.604907,NaN,0.321158,0.272378,8.905333,0.145724,55.050679,2.199299,...,9.794704,2.647823,1.247130,1.054924,2924.587759,3.343896,0.495931,4.463380,0.328686,1.972117
min,3.000000,100001.000000,200001.000000,NaN,0.000000,0.000000,4.000000,0.003455,15.000000,0.000000,...,0.200000,0.000000,15.000000,0.000000,-2600.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13373.000000,124884.000000,224961.000000,NaN,0.000000,0.000000,25.000000,0.048012,52.000000,3.000000,...,19.000000,1.000000,35.666667,2.000000,1023.000000,1.000000,0.000000,0.000000,0.000000,6.000000
50%,26928.000000,149941.000000,250147.000000,NaN,0.000000,0.000000,31.000000,0.097783,65.000000,6.000000,...,26.000000,1.000000,36.111111,2.000000,1680.000000,1.000000,0.000000,0.000000,0.000000,6.000000
75%,60667.000000,175004.000000,275131.000000,NaN,0.000000,0.000000,37.000000,0.188911,77.000000,6.000000,...,30.000000,6.000000,37.388891,4.000000,2559.000000,5.000000,1.000000,9.000000,0.000000,6.000000


In [17]:
df_scores_hos = df_adult_dropped[['age_score', 'preiculos_score',\
                      'gcs_score', 'heartrate_score', 'meanbp_score', \
                 'resprate_score', 'temp_score', 'urineoutput_score', 'mechvent_score',\
                      'electivesurgery_score', 'hospital_expire_flag']].copy()

In [20]:
y = df_scores_hos.pop('hospital_expire_flag')

In [21]:
y

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
12       1
14       0
17       0
18       0
19       1
20       0
21       0
22       1
23       0
24       1
25       0
27       0
28       0
29       0
30       0
31       0
32       0
36       0
37       0
38       0
        ..
61493    0
61495    0
61496    0
61497    0
61498    0
61499    0
61500    0
61503    0
61505    0
61506    0
61509    0
61510    0
61512    0
61513    0
61515    0
61516    0
61517    0
61518    1
61519    0
61520    0
61521    0
61522    0
61523    0
61524    0
61525    0
61526    0
61527    0
61529    0
61530    0
61531    0
Name: hospital_expire_flag, Length: 48785, dtype: int64

In [22]:
X = df_scores_hos.copy()

In [23]:
X.shape

(48785, 10)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=.33,
                                                    random_state=0,
                                                   stratify=y)

In [26]:
# Train and fit model                                                   
rf = RandomForestClassifier(random_state=0,n_jobs=-1)

rf.fit(X_train, y_train)
                                     
# Test Prediction
pred = rf.predict(X_test)
print('Accuracy score: {:.3}'.format(rf.score(X_test, y_test)))

Accuracy score: 0.873


In [34]:
y_predict_proba = rf.predict_proba(X_test)

In [31]:
y_pred = rf.predict(X_test)

In [32]:
roc_auc_score(y_test, y_pred)

0.60032917377382766

In [39]:
y_proba = y_predict_proba[:,1]

In [40]:
roc_auc_score(y_test, y_proba)

0.7274451813448245

In [41]:
rf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)